In [53]:
from rdflib import Graph, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery
import glob
import gzip
from tqdm import tqdm

dir_ = '/data/pj20/compound/general/'

In [54]:
import pickle
with open('./cid_set.pkl', 'rb') as f:
    cid_set = pickle.load(f)

# import json

# smile2cid = {}
# with open('./smile2cid.json', 'r') as f:
#     smile2cid.update(json.load(f))

# icd2smile = {value: key for key, value in smile2cid.items() if value != 'None' and value != None}
# part_cid = set()
# for i in tqdm(icd2smile.keys()):
#     part_cid.add(str(i))

# cid_set.update(part_cid)

In [55]:
len(cid_set)

62174

In [56]:
from tqdm import tqdm
from collections import defaultdict

def retrieve(triple_strs, relation):
    triple_set = set()
    t_buffer = defaultdict(list)
    h_buffer = defaultdict(list)
    for triple_str in tqdm(triple_strs):
        if ' ,\n\t\t' in triple_str:
            items = triple_str.split(' ,\n\t\t')
            hr_t0 = items[0]
            h, r, t0 = hr_t0.split('\t')
            h = h.replace('compound:CID', '')
            t0 = t0.replace('compound:CID', '')

            if h in cid_set and t0 in cid_set:
                triple_set.add((h, relation,t0))

            elif h in cid_set and t0 not in cid_set:
                t_buffer[t0].append(h)
            
            elif h not in cid_set and t0 in cid_set:
                h_buffer[h].append(t0)


            tail_num = len(items) - 1
            for tail in range(tail_num):
                t = items[tail+1].replace('compound:CID', '')
                if h in cid_set and t in cid_set:
                    triple_set.add((h, relation, t))

                elif h in cid_set and t not in cid_set:
                    t_buffer[t].append(h)
                
                elif h not in cid_set and t in cid_set:
                    h_buffer[h].append(t)
        else:
            try:
                items = triple_str.split('\t')
                h = items[0].replace('compound:CID', '')
                t = items[2].replace('compound:CID', '')
                if h in cid_set and t in cid_set:
                    triple_set.add((h, relation, t))

                elif h in cid_set and t not in cid_set:
                    t_buffer[t].append(h)
                
                elif h not in cid_set and t in cid_set:
                    h_buffer[h].append(t)
            except:
                continue
                
    return triple_set, h_buffer, t_buffer

***=============================== compound2component ===============================***

In [57]:
with open(dir_ + 'pc_compound2component.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [58]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_component")

100%|██████████| 23769818/23769818 [00:40<00:00, 583799.01it/s] 


In [59]:
cnt = 0

for key in h_buffer.keys():
    if len(h_buffer[key]) > 30:
        cnt+=1
        for t in h_buffer[key]:
            triple_set.add((key, 'has_component', t))
print(cnt, len(triple_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_component.txt', 'w') as f:
    f.write(out_str)

511 27341


***=============================== compound2parent ===============================***

In [60]:
with open(dir_ + 'pc_compound2parent.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [61]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_parent")

100%|██████████| 6943268/6943268 [00:05<00:00, 1239920.57it/s]


In [62]:
cnt = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 2:
        cnt+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'has_parent', key))
print(cnt, len(triple_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_parent.txt', 'w') as f:
    f.write(out_str)

44 5223


***=============================== compound2stereoisomer ===============================***

In [63]:
with open(dir_ + 'pc_compound2stereoisomer_000001.ttl', 'r') as f:
    lines = f.read()

triple_strs_1 = lines.split(' .\n')[2:]

with open(dir_ + 'pc_compound2stereoisomer_000002.ttl', 'r') as f:
    lines = f.read()

triple_strs_2 = lines.split(' .\n')[2:]

with open(dir_ + 'pc_compound2stereoisomer_000003.ttl', 'r') as f:
    lines = f.read()

triple_strs_3 = lines.split(' .\n')[2:]

triple_strs = triple_strs_1 + triple_strs_2 + triple_strs_3

In [64]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_stereoisomer")

100%|██████████| 24904630/24904630 [00:38<00:00, 649312.21it/s] 


In [65]:
cnt_t = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 10:
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'has_stereoisomer', key))
        cid_set.add(key)

cnt_h = 0
for key in h_buffer.keys():
    if len(h_buffer[key]) > 10:
        cnt_h+=1
        for t in h_buffer[key]:
            triple_set.add((key, 'has_stereoisomer', t))
        cid_set.add(key)

print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_stereoisomer.txt', 'w') as f:
    f.write(out_str)

1229 1229 59370 63403


***=============================== compound2same_connectivity ===============================***

In [66]:
with open(dir_ + 'pc_compound2sameconnectivity.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [67]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_same_connectivity")

100%|██████████| 2821372/2821372 [00:02<00:00, 1357019.35it/s]


In [68]:
cnt_t = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 10:
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'has_same_connectivity', key))
        cid_set.add(key)

cnt_h = 0
for key in h_buffer.keys():
    if len(h_buffer[key]) > 10:
        cnt_h+=1
        for t in h_buffer[key]:
            triple_set.add((key, 'has_same_connectivity', t))
        cid_set.add(key)

print(cnt_t, cnt_h, len(triple_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_connectivity.txt', 'w') as f:
    f.write(out_str)

284 284 85538


***=============================== compound2isotopologue ===============================***

In [69]:
with open(dir_ + 'pc_compound2isotopologue.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [70]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_isotopologue")

100%|██████████| 368953/368953 [00:01<00:00, 202670.62it/s]


In [71]:
cnt_t = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 3:
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'has_isotopologue', key))
        cid_set.add(key)

cnt_h = 0
for key in h_buffer.keys():
    if len(h_buffer[key]) > 3:
        cnt_h+=1
        for t in h_buffer[key]:
            triple_set.add((key, 'has_isotopologue', t))
        cid_set.add(key)

print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_isotopologue.txt', 'w') as f:
    f.write(out_str)

51 51 4436 63738


***=============================== compound2drugproduct ===============================***

In [72]:
with open(dir_ + 'pc_compound2drugproduct.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [73]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="to_drug")

100%|██████████| 17179/17179 [00:00<00:00, 830966.64it/s]


In [74]:
cnt_t = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 2:
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'to_drug', key))


print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    t = t.replace("<http://purl.bioontology.org/ontology/SNOMEDCT/", "SNOMED_").replace(">", "").replace("ns2:", "ns2_")
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_todrug.txt', 'w') as f:
    f.write(out_str)

799 51 3708 63738


***=============================== compound2closematch ===============================***

In [75]:
with open(dir_ + 'pc_compound_closematch.ttl000001.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [76]:
_, _, t_buffer = retrieve(triple_strs, relation="closematch")

100%|██████████| 951092/951092 [00:00<00:00, 1173898.53it/s]


In [77]:
cnt_t = 0

for key in t_buffer.keys():
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'closematch', key))


print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    t = t.replace(":", "_")
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_closematch.txt', 'w') as f:
    f.write(out_str)

39241 51 42949 63738


***=============================== compound2type ===============================***

In [78]:
with open(dir_ + 'pc_compound_type.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [79]:
_, _, t_buffer = retrieve(triple_strs, relation="type")

100%|██████████| 151979/151979 [00:00<00:00, 1095910.37it/s]


In [80]:
cnt_t = 0

for key in t_buffer.keys():
    if len(t_buffer[key]) > 2:
        cnt_t+=1
        for h in t_buffer[key]:
            triple_set.add((h, 'type', key))


print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    t = t.replace("<http://purl.bioontology.org/ontology/SNOMEDCT/", "SNOMED_").replace(">", "").replace(":", "_")
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_totype.txt', 'w') as f:
    f.write(out_str)

1189 51 48360 63738


***=============================== compound2descriptor ===============================***

In [81]:
from tqdm import tqdm

triple_strs = []

for i in tqdm(range(1, 161)):
    if len(str(i)) == 1:
        idx = "00" + str(i)
    if len(str(i)) == 2:
        idx = "0" + str(i)
    if len(str(i)) == 3:
        idx = str(i)
    
    file_name = dir_ + f'pc_compound2descriptor_000{idx}.ttl'

    with open(file_name, 'r') as f:
        lines = f.read()

    triple_strs += lines.split(' .\n')[2:]

100%|██████████| 160/160 [02:52<00:00,  1.08s/it]


In [82]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="has_descriptor")

100%|██████████| 114013741/114013741 [15:17<00:00, 124266.80it/s]


In [83]:
cnt_t = 0

for key in t_buffer.keys():
    for h in t_buffer[key]:
        triple_set.add((h, 'has_descriptor', key))


print(cnt_t, cnt_h, len(triple_set), len(cid_set))

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_descriptor.txt', 'w') as f:
    f.write(out_str)

0 51 1456589 63738


***=============================== compound2pathway ===============================***

In [84]:
with open(dir_ + 'pc_pathway.ttl', 'r') as f:
    lines = f.read()

triple_strs = lines.split(' .\n')[2:]

In [85]:
triple_set, h_buffer, t_buffer = retrieve(triple_strs, relation="in_pathway")

100%|██████████| 494606/494606 [00:01<00:00, 468974.82it/s]


In [86]:
cnt_t = 0

triple_set = set()
for key in h_buffer.keys():
    tmp_set = set()
    for cid in h_buffer[key]:
        if cid in cid_set:
            tmp_set.add((cid, 'in_pathway', key.replace('pathway:', '')))
        if len(tmp_set) > 8:
            triple_set.update(tmp_set)

out_str = ""

for triple in triple_set:
    h, r, t = triple
    out_str += h +'\t' + r  + '\t' + t +'\n'

with open('./compound_pathway.txt', 'w') as f:
    f.write(out_str)